In [1]:
import json
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Load data
with open('postm.json', 'r') as f:
    data = json.load(f)

# Extract data
date_range = data['date_range']
feature_importance = data['feature_importance']
anomaly_overall = data['anomaly_detection_overall']
plot_data = data['plot_data']

print("=" * 80)
print(f"📅 DATE RANGE: {date_range['start']} to {date_range['end']}")
print(f"📊 TOTAL SEQUENCES: {date_range['total_sequences']}")
print(f"🚨 ANOMALIES FOUND: {date_range['anomalies_found']} ({date_range['anomaly_percentage']}%)")
print("=" * 80)

# ============================================================================
# PLOT 1: Overall Anomaly Detection
# ============================================================================

fig1 = go.Figure()

timestamps = anomaly_overall['timestamps']
errors = anomaly_overall['reconstruction_errors']
flags = anomaly_overall['anomaly_flags']
threshold_lower = anomaly_overall['threshold_lower']
threshold_upper = anomaly_overall['threshold_upper']

# Separate normal and anomaly points
normal_indices = [i for i, flag in enumerate(flags) if not flag]
anomaly_indices = [i for i, flag in enumerate(flags) if flag]

# Normal points (blue)
if normal_indices:
    fig1.add_trace(go.Scatter(
        x=[timestamps[i] for i in normal_indices],
        y=[errors[i] for i in normal_indices],
        mode='markers',
        marker=dict(size=4, color='blue'),
        name='Normal',
        hovertemplate='%{x}<br>Error: %{y:.2f}<extra></extra>'
    ))

# Anomaly points (red)
if anomaly_indices:
    fig1.add_trace(go.Scatter(
        x=[timestamps[i] for i in anomaly_indices],
        y=[errors[i] for i in anomaly_indices],
        mode='markers',
        marker=dict(size=8, color='red', symbol='diamond'),
        name='Anomaly',
        hovertemplate='%{x}<br>Error: %{y:.2f}<extra></extra>'
    ))

# Threshold lines
fig1.add_hline(y=threshold_upper, line_dash="dash", line_color="red", 
               annotation_text=f"Upper ({threshold_upper})")
fig1.add_hline(y=threshold_lower, line_dash="dash", line_color="green",
               annotation_text=f"Lower ({threshold_lower})")

fig1.update_layout(
    title=f"Overall Anomaly Detection - {date_range['anomalies_found']} anomalies found",
    xaxis_title="Timestamp",
    yaxis_title="Reconstruction Error",
    height=600,
    hovermode='x unified'
)

fig1.show()

# ============================================================================
# PLOT 2: Feature Importance (SHAP Rankings)
# ============================================================================

if date_range['anomalies_found'] > 0:
    # Sort by rank
    sorted_features = sorted(feature_importance, key=lambda x: x['rank'])
    
    feature_names = [f['feature_name'] for f in sorted_features]
    shap_scores = [f['shap_score'] for f in sorted_features]
    
    fig2 = go.Figure()
    
    fig2.add_trace(go.Bar(
        y=feature_names[::-1],  # Reverse to show highest at top
        x=shap_scores[::-1],
        orientation='h',
        marker=dict(
            color=shap_scores[::-1],
            colorscale='Reds',
            showscale=True
        ),
        text=[f"{score:.4f}" for score in shap_scores[::-1]],
        textposition='auto',
        hovertemplate='%{y}<br>SHAP Score: %{x:.4f}<extra></extra>'
    ))
    
    fig2.update_layout(
        title="Feature Importance (SHAP Rankings) - Most Problematic Features",
        xaxis_title="SHAP Score",
        yaxis_title="Feature",
        height=600,
        showlegend=False
    )
    
    fig2.show()
    
    print("\n📊 TOP 5 CONTRIBUTING FEATURES:")
    print("=" * 80)
    for i, feature in enumerate(sorted_features[:5], 1):
        print(f"{i}. {feature['feature_name']}: {feature['shap_score']:.4f}")
    print("=" * 80)
else:
    print("\n⚠️  No anomalies found - no SHAP rankings to display")

# ============================================================================
# PLOT 3: Per-Feature Actual vs Reconstructed (Top 3 features)
# ============================================================================

if date_range['anomalies_found'] > 0:
    # Get top 3 features
    top_3_features = sorted_features[:3]
    
    fig3 = make_subplots(
        rows=3, cols=1,
        subplot_titles=[f"{f['feature_name']} (Rank {f['rank']})" for f in top_3_features],
        vertical_spacing=0.1
    )
    
    for idx, feature in enumerate(top_3_features, 1):
        feature_name = feature['feature_name']
        
        # Find this feature in plot_data
        feature_plot = next((f for f in plot_data if f['feature_name'] == feature_name), None)
        
        if feature_plot:
            timestamps_feat = feature_plot['timestamps']
            actual = feature_plot['actual']
            reconstructed = feature_plot['reconstructed']
            threshold_lower_feat = feature_plot['threshold_lower']
            threshold_upper_feat = feature_plot['threshold_upper']
            
            # Actual values
            fig3.add_trace(
                go.Scatter(
                    x=timestamps_feat, y=actual,
                    mode='markers',
                    marker=dict(size=4, color='blue'),
                    name='Actual',
                    showlegend=(idx==1),
                    hovertemplate='%{x}<br>Actual: %{y:.2f}<extra></extra>'
                ),
                row=idx, col=1
            )
            
            # Reconstructed values
            fig3.add_trace(
                go.Scatter(
                    x=timestamps_feat, y=reconstructed,
                    mode='markers',
                    marker=dict(size=4, color='red'),
                    name='Reconstructed',
                    showlegend=(idx==1),
                    hovertemplate='%{x}<br>Reconstructed: %{y:.2f}<extra></extra>'
                ),
                row=idx, col=1
            )
            
            # Threshold lines
            if threshold_upper_feat:
                fig3.add_hline(y=threshold_upper_feat, line_dash="dash", line_color="orange",
                              row=idx, col=1)
            if threshold_lower_feat:
                fig3.add_hline(y=threshold_lower_feat, line_dash="dash", line_color="green",
                              row=idx, col=1)
    
    fig3.update_layout(
        title="Top 3 Contributing Features - Actual vs Reconstructed",
        height=900,
        hovermode='x unified'
    )
    
    fig3.update_xaxes(title_text="Timestamp")
    fig3.update_yaxes(title_text="Value")
    
    fig3.show()

print("\n✅ ALL PLOTS GENERATED!")

📅 DATE RANGE: 2024-01-01 to 2025-12-31
📊 TOTAL SEQUENCES: 1247
🚨 ANOMALIES FOUND: 13 (1.04%)



📊 TOP 5 CONTRIBUTING FEATURES:
1. ME_NO2_CYL_LINER_PP_SIDE_TEMP: 1.0000
2. M_E_NO_1_T_C_EXH_GAS_OUT_TEMP: 0.6123
3. ME_RPM: 0.5896
4. ME_Tc_rpm: 0.5018
5. BOILER_EXH_GAS_OUT_M_E_TEMP_H: 0.4636



✅ ALL PLOTS GENERATED!
